In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sko.GA import GA
from sko.PSO import PSO
import time
from keras.models import Model, load_model
from sko.GA import GA
from sko.DE import DE
from sklearn import tree
%matplotlib
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.45
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('database.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'A_c',#0
                      'A_a',#1
                      'L_cl',#2
                      'i0',#3
                      'L-PEM',#4
                      'L_GDL',#5
                      'epsl_cl',#6
                      'epsp_cl',#7
                      'T_cc',#8
                      'RH_cc',#9
                      'pressure',#10
                        'Volt0.9',
                        'Volt0.8',
                        'Volt0.7',#13
                        'Volt0.6',
                        'Volt0.5',
                        'Volt0.4',
                        'Volt0.3',
                        'Volt0.2',
                        'Volt0.1',
                        'Volt0'
                        ]]
###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:11]
raw_output=standardized_data.iloc[:,11:]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########fix random seed for reproducability##########
seed=1
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,random_state=seed)
raw_input_global=raw_data.iloc[:,0:11]
raw_output_global=raw_data.iloc[:,11:]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})

In [ ]:
model=load_model('ANN_CURVE.h5')

In [ ]:
def processing_PRINT(thickness):
    median=standardized_data.median()
    raw_median=raw_data.median()
    user_input1=raw_median[0:11]    
    dict_result={}    
    user_input1.iloc[0]=1.872e7
    user_input1.iloc[1]=1.872e7
    user_input1.iloc[2]=thickness
    user_input1.iloc[3]=24.5
    user_input1.iloc[4]=23
    user_input1.iloc[5]=125
    user_input1.iloc[6]=0.19
    user_input1.iloc[7]=0.64
    user_input1.iloc[8]=353.15
    user_input1.iloc[9]=1
    user_input1.iloc[10]=300000
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:11])/np.std(raw_data,axis=0)[0:11]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0).T.values+np.mean(raw_output_global,axis=0).T.values
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
#     print(x_prediction_maximum_power_ann)
    x_07=x_prediction_ann[2]
    dict_result.update({x_07:{
                                'thickness':thickness}})
    print(x_07)

    return x_07, dict_result
import time
time_start = time.time()
for i in np.arange(1,15,0.01):
    processing_PRINT(i)

time_end = time.time()
print('Time cost of grid search= %fs' % (time_end - time_start))

In [ ]:
%matplotlib
def processing_curve_ic(a,thickness,epsl_cl,epsp_cl):
    median=standardized_data.median()
    raw_median=raw_data.median()
    user_input1=raw_median[0:11]    
    dict_result={}    
    user_input1.iloc[0]=a
    user_input1.iloc[1]=a
    user_input1.iloc[2]=thickness
    user_input1.iloc[3]=24.5
    user_input1.iloc[4]=23
    user_input1.iloc[5]=125
    user_input1.iloc[6]=epsl_cl
    user_input1.iloc[7]=epsp_cl
    user_input1.iloc[8]=353.15
    user_input1.iloc[9]=1
    user_input1.iloc[10]=300000
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:11])/np.std(raw_data,axis=0)[0:11]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0).T.values+np.mean(raw_output_global,axis=0).T.values
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
    return x_prediction_ann[0:]
a_list=[18720000,15709969.79, 6514522.822, 17476538.06, 17958179.58]
thickness_list=[5, 5.627, 9.64, 4.795, 4.878]
epsl_list=[0.190349291, 0.212508265, 0.921469597, 0.11578424, 0.151752202]
epsp_list=[0.640081927, 0.636817502, 0.01, 0.707397426, 0.674438058]
voltage_list=[0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0]
for i in range (0,5):
    print(processing_curve_ic(a_list[i],thickness_list[i],epsl_list[i],epsp_list[i]))
    print(get_power_density(processing_curve_ic(a_list[i],thickness_list[i],epsl_list[i],epsp_list[i]),voltage_list))
